In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import plotly.offline as py
import cufflinks as cf

cf.set_config_file(offline=True)

<br/><br/><br/>

---

# (Demo 1) Canonicalization

In [ ]:
with open('county_and_state.csv') as f:
    county_and_state = pd.read_csv(f)
    
with open('county_and_population.csv') as f:
    county_and_pop = pd.read_csv(f)    

Suppose we'd like to join these two tables. Unfortunately, we can't, because the strings representing the county names don't match, as seen below.

In [ ]:
county_and_state

In [ ]:
county_and_pop

 Before we can join them, we'll do what I call **canonicalization**.

Canonicalization: A process for converting data that has more than one possible representation into a "standard", "normal", or canonical form (definition via Wikipedia).

In [ ]:
def canonicalize_county(county_name):
    return (
        county_name
        .lower()               # lower case
        .replace(' ', '')      # remove spaces
        .replace('&', 'and')   # replace &
        .replace('.', '')      # remove dot
        .replace('county', '') # remove county
        .replace('parish', '') # remove parish
    )

In [ ]:
county_and_pop['clean_county'] = county_and_pop['County'].map(canonicalize_county)
county_and_state['clean_county'] = county_and_state['County'].map(canonicalize_county)

Displaying the tables inline.  Here we use the builtin display function from iPython. 

In [ ]:
display(county_and_pop)
display(county_and_state)

## Pandas String Operations

If this were a large dataframe it would have been more efficient to use pandas `str` operations.

In [ ]:
def canonicalize_county_series(count_name_series):
    return (
        count_name_series
        .str.lower()               # lower case
        .str.replace(' ', '')      # remove spaces
        .str.replace('&', 'and')   # replace &
        .str.replace('.', '')      # remove dot
        .str.replace('county', '') # remove county
        .str.replace('parish', '') # remove parish
    )

In [ ]:
county_and_pop['clean_county'] = canonicalize_county_series(county_and_pop['County'])
county_and_state['clean_county'] = canonicalize_county_series(county_and_state['County'])

In [ ]:
display(county_and_pop)
display(county_and_state)

Finishing the join

In [ ]:
county_and_pop.merge(county_and_state,
                     left_on = 'clean_county', right_on = 'clean_county')

<br/><br/><br/>

---

# (Demo 2) Processing Data from a Text Log Using Basic Python

In [ ]:
with open('log.txt', 'r') as f:
    log_lines = f.readlines()

In [ ]:
log_lines

Suppose we want to extract the day, month, year, hour, minutes, seconds, and timezone. Looking at the data, we see that these items are not in a fixed position relative to the beginning of the string. 

That is, slicing by some fixed offset isn't going to work.

In [ ]:
log_lines[0][20:31]

In [ ]:
log_lines[1][20:31]

Instead, we'll need to use some more sophisticated thinking. Let's focus on only the first line of the file.

In [ ]:
first = log_lines[0]
first

When working with text it helps to grab a sample line.  However, you will want to check that your rule generalizes.

In [ ]:
pertinent = first.split("[")[1].split(']')[0]
pertinent

In [ ]:
day, month, rest = pertinent.split('/')
print("day =", day)
print("month =", month)
print("rest =", rest)

In [ ]:
year, hour, minute, rest = rest.split(':')
print("year =", year)
print("hour =", hour)
print("minute =", minute)
print("rest =", rest)

In [ ]:
seconds, time_zone = rest.split(' ')
day, month, year, hour, minute, seconds, time_zone

### Converting this into Dataframe Operations

Suppose this was a column of text in a dataframe:

In [ ]:
df = pd.DataFrame({"text": log_lines})
df

It is often helpful to extract the series to get better tab completion

In [ ]:
ts = df['text']

In [ ]:
ts.str.split("[")

Write the command on multiple lines:

In [ ]:
(ts
     .str.split("[").str[1]
     .str.split("]").str[0]
     .str.split("/")
)

In [ ]:
(ts
     .str.split("[").str[1]
     .str.split("]").str[0]
     .str.split("/", expand=True)
)

In [ ]:
df2 = (ts
       .str.split("[").str[1]
       .str.split("]").str[0]
       .str.split("/", expand=True)
       .rename(columns={0:"Day", 1:"Month", 2:"Rest"})
)
df2

In [ ]:
ts2 = df2["Rest"]
ts2

In [ ]:
df3 = (
    ts2.str.split(" ").str[0]
        .str.split(":",expand=True)
        .rename(columns={0:"Year", 1:"Hour", 2: "Minute", 3:"Second"})
)
df3

Combining everything.

In [ ]:
df2[['Day', 'Month']].join(df3)

## A Taste of Regex 

A much more sophisticated but common approach is to extract the information we need using a regular expression. See today's lecture slidesfor more on regular expressions.

In [ ]:
import re
pattern = r'\[(\d+)/(\w+)/(\d+):(\d+):(\d+):(\d+) (.+)\]'
day, month, year, hour, minute, second, time_zone = re.findall(pattern, first)[0]
year, month, day, hour, minute, second, time_zone

Note: We can return the results as a Series:

In [ ]:
cols = ["Day", "Month", "Year", "Hour", "Minute", "Second", "Time Zone"]
def log_entry_to_series(line):
    return pd.Series(re.findall(pattern, line)[0], index = cols)

log_entry_to_series(first)

And using this function we can create a DataFrame of all the time information.

In [ ]:
log_info = pd.DataFrame(columns=cols)

for line in log_lines:
    log_info = log_info.append(log_entry_to_series(line), ignore_index = True)

log_info

### Using Pandas String Operations 

A more efficient way to do these transformations would be:

In [ ]:
ts.str.extract(pattern)

In [ ]:
(
    ts.str.extract(pattern, expand=True)
        .rename(columns={
            0:"Day", 1:"Month", 2:"Year",
            3:"Hour", 4:"Minute", 5:"Second",
            6:"TimeZone"
        })
        .astype({"Day":int, "Year":int, "Hour":int, "Second":int})
)

<br/><br/><br/>

--- 

# Regular Expression From Lecture

Fill in the regex below so that after code executes, day is “26”, month is “Jan”, and year is “2014”. 

<details>
    <summary>Solution</summary>
pattern = r"\[(\d+)/(\w+)/(\d+):"
    
</details>

In [ ]:
log_lines[0]

In [ ]:
pattern = r"fill me in"
matches = re.findall(pattern, log_lines[0])
print(matches)
# day, month, year = matches[0]
#day, month, year

## Real World Example #1: Restaurant Data

In this example, we will show how regexes can allow us to track quantitative data across categories defined by the appearance of various text fields.

In this example we'll see how the presence of certain keywords can affect quantitative data, e.g. how do restaurant health scores vary as a function of the number of violations that mention "vermin"?

In [ ]:
vio = pd.read_csv('violations.csv.bz2', header=0, names=['id', 'date', 'desc'])
desc = vio['desc']
vio.head()

In [ ]:
counts = desc.value_counts()
counts

In [ ]:
# Hmmm...
counts.tail()

In [ ]:
#Use regular expressions to cut out the extra info in square braces.
vio['clean_desc'] = (
    vio['desc']
             .str.replace('\s*\[[^\]]*\]$', '')
             .str.strip()
             .str.lower())
print("Before:", "#" + vio['desc'].iloc[-1] + "#")
print("After:", "#" + vio['clean_desc'].iloc[-1] + "#")

<br/><br/><br/>

---

# Extracting interesting features

In [ ]:
vio['clean_desc'].value_counts().head() 

In [ ]:
#use regular expressions to assign new features for the presence of various keywords
with_features = (vio
 .assign(is_clean     = vio['clean_desc'].str.contains('clean|sanit'))
 .assign(is_high_risk = vio['clean_desc'].str.contains('high risk'))
 .assign(is_vermin    = vio['clean_desc'].str.contains('vermin'))
 .assign(is_surface   = vio['clean_desc'].str.contains('wall|ceiling|floor|surface'))
 .assign(is_human     = vio['clean_desc'].str.contains('hand|glove|hair|nail'))
 .assign(is_permit    = vio['clean_desc'].str.contains('permit|certif'))
)
with_features.head()

Exploring aggregate features for each inspection (business id and date). 

In [ ]:
count_features = (with_features
 .groupby(['id', 'date'])
 .sum()
 .reset_index()
)
count_features.iloc[255:260, :]

In [ ]:
count_features[count_features['is_vermin'] > 1]

In [ ]:
#use a new pandas feature called "melt" that we won't describe in any detail
#the granularity of the resulting frame is a violation type in a given inspection
broken_down_by_violation_type = pd.melt(count_features, id_vars=['id', 'date'],
            var_name='feature', value_name='num_vios')
broken_down_by_violation_type.sort_values(["id", "date"]).head(13)

In [ ]:
#read in the scores
ins = pd.read_csv('inspections.csv.bz2',
                  header=0,
                  usecols=[0, 1, 2],
                  names=['id', 'score', 'date'])
ins.head()

In [ ]:
#join scores with the table broken down by violation type
violation_type_and_scores = (
    broken_down_by_violation_type
    .merge(ins, left_on=['id', 'date'], right_on=['id', 'date'])
)
violation_type_and_scores.head(12)

In [ ]:
sns.catplot(x='num_vios', y='score',
               col='feature', col_wrap=2,
               kind='box',
               data=violation_type_and_scores)

Above we see, for example, that if a restaurant inspection involved 2 violation with the keyword "vermin", the average score for that inspection would be a little bit below 80.

## Text Processing Example 2: Police Data

In this example, we will apply string processing to the process of data cleaning and exploratory data analysis.

### Getting the Data

The city of Berkeley maintains an [Open Data Portal](https://data.cityofberkeley.info/) for citizens to access data about the city.  We will be examining [Call Data](https://data.cityofberkeley.info/Public-Safety/Berkeley-PD-Calls-for-Service/k2nh-s5h5).

<img src="calls_desc.png" width=800px />



In [ ]:
import ds100_utils

calls_url = 'https://data.cityofberkeley.info/api/views/k2nh-s5h5/rows.csv?accessType=DOWNLOAD'
calls_file = ds100_utils.fetch_and_cache(calls_url, 'calls.csv')
calls = pd.read_csv(calls_file, warn_bad_lines=True)
calls.head()

How many records did we get?

In [ ]:
len(calls)

What does an example `Block_Location` value look like?

In [ ]:
print(calls['Block_Location'].iloc[15])

### Preliminary observations on the data?

1. `EVENTDT` -- Contains the incorrect time
1. `EVENTTM` -- Contains the time in 24 hour format (What timezone?)
1. `CVDOW` -- Encodes the day of the week (see data documentation).
1. `InDbDate` -- Appears to be correctly formatted and appears pretty consistent in time.
1. **`Block_Location` -- a multi-line string that contains coordinates.**
1. `BLKADDR` -- Appears to be the address in `Block Location`.
1. `City` and `State` seem redundant given this is supposed to be the city of Berkeley dataset.

### Extracting locations

The block location contains geographic coordinates. Let's extract them.

In [ ]:
calls['Block_Location'][15]

In [ ]:
calls_lat_lon = (
    calls['Block_Location']
    .str.extract("\((\d+\.\d+)\, (-\d+\.\d+)\)")
)
calls_lat_lon.columns = ['Lat', 'Lon']
calls_lat_lon.head(10)

How many records have missing values?

In [ ]:
calls_lat_lon.isnull().sum()

Examine the missing values.

In [ ]:
calls[calls_lat_lon.isnull().any(axis=1)]['Block_Location'].head(10)

Join in the extracted values.

In [ ]:
calls['Lat'] = calls_lat_lon['Lat']
calls['Lon'] = calls_lat_lon['Lon']
calls.head()

## Examining Location information

Let's examine the geographic data (latitude and longitude).  Recall that we had some missing values.  Let's look at the behavior of these missing values according to crime type.

In [ ]:
missing_lat_lon = calls[calls[['Lat', 'Lon']].isnull().any(axis=1)]
(missing_lat_lon['CVLEGEND'].value_counts() 
 / calls['CVLEGEND'].value_counts()
).sort_values(ascending=False).iplot(kind="barh");

Now, let's make a crime map.

In [ ]:
# you may need to install the folium package for this to work
# to do this, uncomment the line below and run it, this might take 
# more than 5 minutes. It'll probably say "solving environment: " for
# a long time.
#!conda --debug install -y -c conda-forge folium 

In [ ]:
import folium
import folium.plugins

SF_COORDINATES = (37.87, -122.28)
sf_map = folium.Map(location=SF_COORDINATES, zoom_start=13)
locs = calls[['Lat', 'Lon']].astype('float').dropna().values
heatmap = folium.plugins.HeatMap(locs.tolist(), radius=10)
sf_map.add_child(heatmap)

### Questions

1. Is campus really the safest place to be?
1. Why are all the calls located on the street and at often at intersections?


In [ ]:
import folium.plugins

# Need to sample the data otherwise browser has issues.
locations = calls[['Lat', 'Lon', "OFFENSE"]].sample(1000)

cluster = folium.plugins.MarkerCluster()
for _, r in locations.dropna().iterrows():
    cluster.add_child(
        folium.Marker([float(r["Lat"]), float(r["Lon"])], tooltip=r['OFFENSE']))
    
sf_map = folium.Map(location=SF_COORDINATES, zoom_start=13)
sf_map.add_child(cluster)
sf_map